# Novo kernel C++

Este novo kernel inclui um facilitador em que você pode escrever comando em C++ sem a função `main`. Este é um facilitador do Jupyter. Em um código C++ completo você teria que escrever a função.

Substituiremos o `printf` por uma outra notação usando o `std::cout`. O `std::cout` representa a saída e você usa dois sinais de menor `<<` para canalizar o que quer mostrar no console para ela. Como o exemplo a seguir.

Por enquanto, não se preocupe em entender o mecanismo de funcionamento, que será apresentado no futuro.

Todas as células são partes de um mesmo programa. Desse modo a importação do `iostream` feita a seguir servirá para as demais células.

In [1]:
#include <iostream>

std::cout << "o dinossauro pulou na lama";

o dinossauro pulou na lama

## Usando o `std::cout`

Você pode concatenar uma variável em uma string e o C++ resolve tudo para você:

In [2]:
int var = 5;
std::cout << "Valor da variavel: " << var;

Valor da variavel: 5

## Próxima linha

Também pode usar `std::endl` para indicar próxima linha:

In [3]:
std::cout << "linha 1" << std::endl << "linha 2";

linha 1
linha 2

# Classe

Inicialmente, vamos pensar em classe como uma estratégia especial de modularização. Adiante, veremos que ela é mais do que isso.

Vamos retomar a struct construída no bloco anterior contendo funções:
```c
typedef struct Emprestimo {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
    float (*proximaParcela)(struct Emprestimo*);
} Emprestimo;
```

Substituiremos a struct por uma nova construção que chamaremos de classe. A classe consegue ir além da struct pois ela pode literalmente encapsular as próprias funções:

```cpp
class Emprestimo {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
    public:
        Emprestimo(float ps, int pn, float pj) { ... }
        float proximaParcela() { ... }
};
```

Os campos da classe (s, n, j, corrente e p) se chamam `atributos`. Adiante, veremos a importância de chamá-los assim, quando as classes forem associadas à forma como pensamos o mundo.

In [2]:
#include <iostream>

class Emprestimo {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
    public:
        Emprestimo(float ps, int pn, float pj) {
            s = ps;
            n = pn;
            j = pj;
            corrente = 1;
            p = ps;
        }

        float proximaParcela() {
            float retorno = p;
            corrente++;
            if (corrente <= n)
                p = p + (p * (j/100));
            else
                p = 0;
            return retorno;
        }
};

// codigo principal

Emprestimo emprestimo1(200, 5, 1),
           emprestimo2(500, 7, 2);

int i = 1;
float p1 = emprestimo1.proximaParcela();
float p2 = emprestimo2.proximaParcela();
while (p1 > 0 || p2 > 0) {
    if (p1 > 0)
        std::cout << "Emprestimo 1: parcela " << i << " eh " << p1 << std::endl;
    if (p2 > 0)
        std::cout << "Emprestimo 2: parcela " << i << " eh " << p2 << std::endl;
    p1 = emprestimo1.proximaParcela();
    p2 = emprestimo2.proximaParcela();
    i++;
}

Emprestimo 1: parcela 1 eh 200
Emprestimo 2: parcela 1 eh 500
Emprestimo 1: parcela 2 eh 202
Emprestimo 2: parcela 2 eh 510
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.2
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.604
Emprestimo 1: parcela 5 eh 208.121
Emprestimo 2: parcela 5 eh 541.216
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.081


# Objeto - instância da Classe

As funções de uma classe são chamadas de métodos. Diferentemente da struct, cada método ganha um contexto dentro da classe, ou seja, ela passa a ter acesso aos valores dos atributos da instância da classe a que pertence.

Veja como o método `proximaParcela` se refere aos atributos:

```cpp
float proximaParcela() {
    int retorno = p;
    corrente++;
    if (corrente <= n)
        p = p + (p * (j/100));
    else
        p = 0;
    return retorno;
}
```

Veja como a função `proximaParcela` da struct fazia a mesma coisa:
```c
float proximaParcela(struct Emprestimo *umEmprestimo) {
    float retorno = umEmprestimo->p;
    if (umEmprestimo->corrente < umEmprestimo->n)
        umEmprestimo->p = umEmprestimo->p + (umEmprestimo->p * (umEmprestimo->j/100));
    else
        umEmprestimo->p = 0;
    (umEmprestimo->corrente)++;
    return retorno;
}
```

Na struct foi necessário se falar explicitamente que a função está trabalhando com a struct `umEmprestimo` recebida como parâmetro.

Como o método da classe sabe com quem está trabalhando?

## Construindo um objeto

Há um método especial chamado construtor que recebe o mesmo nome da classe. Nesse caso, o método construtor se chama `Emprestimo`. Quando você declara uma variável do tipo `Emprestimo` esse método é chamado implicitamente. Por isso você teve que informar parâmetros na hora que declarou.

```cpp
Emprestimo emprestimo1(200, 5, 1);
```

O construtor, ao ser chamado, criará um objeto. Cada objeto pode ser pensado como instância de uma classe, de forma semelhante ao modo como você cria uma instância de uma struct. Ou seja, será criado um espaço em memória específico para os atributos deste objeto. Chamaremos este espaço de atributos da instância, ou atributos do objeto.

Note que o construtor está dando valores aos atributos do objeto.

```cpp
Emprestimo(float ps, int pn, float pj) {
    s = ps;
    n = pn;
    j = pj;
    corrente = 1;
    p = ps;
}
```

Os atributos de um objeto são reconhecidos automaticamente pelo método no momento de sua ativação. Ou seja, na chamada a seguir, o método `proximaParcela` saberá que terá que manipular os atributos do objeto colocado em `emprestimo1`:

```cpp
float p1 = emprestimo1.proximaParcela();
```

Do mesmo modo, na instrução `proximaParcela` sabe que trabalhará com os atributos colocados em `emprestimo2`. É isso que chamamos de contexto.

```cpp
float p2 = emprestimo2.proximaParcela();
```